# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.65,115.33,54.00,93,0,8.88,AU,1595725903
1,rikitea,-23.12,-134.97,70.48,80,19,19.53,PF,1595725422
2,qaanaaq,77.48,-69.36,43.47,61,0,0.85,GL,1595725387
3,taoudenni,22.68,-3.98,96.87,28,0,6.73,ML,1595725971
4,cape town,-33.93,18.42,59.00,93,75,5.82,ZA,1595725767
...,...,...,...,...,...,...,...,...,...
580,baruun-urt,46.68,113.28,65.53,83,100,7.83,MN,1595726184
581,shu,43.60,73.76,75.00,68,0,3.22,KZ,1595726184
582,moyale,3.52,39.06,62.98,80,1,14.05,KE,1595726185
583,pindiga,9.98,10.93,72.25,92,100,7.27,NG,1595726185


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#store lat and long
locations = weather_data[["Lat", "Lng"]]

#store humidity
humidity = weather_data["Humidity"]

In [4]:
#plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
new_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & (weather_data["Max Temp"] >= 75) & (weather_data["Max Temp"] <= 78)].dropna()

new_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
232,surt,31.21,16.59,75.02,78,0,2.33,LY,1595726050
258,beni suef,29.07,31.10,76.33,40,0,8.70,EG,1595726058
275,gat,31.61,34.76,77.00,62,0,2.08,IL,1595726064
400,loralai,30.37,68.60,76.48,59,0,2.91,PK,1595726120
418,alexandria,31.22,29.96,77.00,78,0,6.93,EG,1595726128
457,miguel pereira,-22.45,-43.47,77.00,69,0,6.93,BR,1595726142
465,murgab,37.50,61.97,77.00,44,0,4.70,TM,1595726145
495,bhadrachalam,17.67,80.88,77.74,90,0,0.74,IN,1595726155
581,shu,43.60,73.76,75.00,68,0,3.22,KZ,1595726184
584,waraseoni,21.75,80.03,77.97,76,0,1.59,IN,1595726185


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = new_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
232,surt,LY,31.21,16.59,
258,beni suef,EG,29.07,31.10,
275,gat,IL,31.61,34.76,
400,loralai,PK,30.37,68.60,
418,alexandria,EG,31.22,29.96,
457,miguel pereira,BR,-22.45,-43.47,
465,murgab,TM,37.50,61.97,
495,bhadrachalam,IN,17.67,80.88,
581,shu,KZ,43.60,73.76,
584,waraseoni,IN,21.75,80.03,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [24]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
# assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={lat},{lng}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city_name} at {lat} , {lng} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    

Retrieving Results for Index 232: surt.
Closest hotel to surt at 31.21 , 16.59 is مصحة مكمداس الطبية الأيوائية سرت 24H HA.
------------
Retrieving Results for Index 258: beni suef.
Closest hotel to beni suef at 29.07 , 31.1 is Anwar El-Sadat School.
------------
Retrieving Results for Index 275: gat.
Closest hotel to gat at 31.61 , 34.76 is קרית גת.
------------
Retrieving Results for Index 400: loralai.
Closest hotel to loralai at 30.37 , 68.6 is Habib Plaza.
------------
Retrieving Results for Index 418: alexandria.
Closest hotel to alexandria at 31.22 , 29.96 is 26th Of July Hotel For Armed Forces.
------------
Retrieving Results for Index 457: miguel pereira.
Closest hotel to miguel pereira at -22.45 , -43.47 is Rancho Renascença.
------------
Retrieving Results for Index 465: murgab.
Closest hotel to murgab at 37.5 , 61.97 is Средняя Школа 18.
------------
Retrieving Results for Index 495: bhadrachalam.
Closest hotel to bhadrachalam at 17.67 , 80.88 is Bhadragiri Hotel.
----------

In [25]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
232,surt,LY,31.21,16.59,مصحة مكمداس الطبية الأيوائية سرت 24H HA
258,beni suef,EG,29.07,31.10,Anwar El-Sadat School
275,gat,IL,31.61,34.76,קרית גת
400,loralai,PK,30.37,68.60,Habib Plaza
418,alexandria,EG,31.22,29.96,26th Of July Hotel For Armed Forces


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))